In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os


In [5]:
train_images_path="/content/drive/MyDrive/All-weapons-data-1/train/images"
train_labels_path="/content/drive/MyDrive/All-weapons-data-1/train/labels"
test_images_path="/content/drive/MyDrive/All-weapons-data-1/test/images"
test_labels_path="/content/drive/MyDrive/All-weapons-data-1/test/labels"
validation_images_path="/content/drive/MyDrive/All-weapons-data-1/valid/images"
validation_labels_path="/content/drive/MyDrive/All-weapons-data-1/valid/labels"

We will first check if all the train , test and valid set have correct corresponding images and labels or not


In [ ]:

def check_image_label_pairs(images_path, labels_path):
    image_names = {os.path.splitext(f)[0] for f in os.listdir(images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
    label_names = {os.path.splitext(f)[0] for f in os.listdir(labels_path) if f.lower().endswith('.txt')}

    missing_labels = image_names - label_names
    missing_images = label_names - image_names

    print(f"Total images: {len(image_names)}")
    print(f"Total labels: {len(label_names)}")
    print("")

    if missing_labels:
        print(f"{len(missing_labels)} images have no label file:")
        for name in sorted(list(missing_labels))[:10]:
            print("  -", name)
        if len(missing_labels) > 10:
            print(f"  ... and {len(missing_labels)-10} more.")
    else:
        print("All images have matching labels.")

    print("")

    if missing_images:
        print(f"{len(missing_images)} labels have no image file:")
        for name in sorted(list(missing_images))[:10]:
            print("  -", name)
        if len(missing_images) > 10:
            print(f"  ... and {len(missing_images)-10} more.")
    else:
        print("All labels have matching images.")


In [9]:
check_image_label_pairs(train_images_path, train_labels_path)

Total images: 7396
Total labels: 7396

All images have matching labels.

All labels have matching images.


In [10]:
check_image_label_pairs(test_images_path, test_labels_path)

Total images: 593
Total labels: 593

All images have matching labels.

All labels have matching images.


In [11]:
check_image_label_pairs(validation_images_path, validation_labels_path)

Total images: 591
Total labels: 591

All images have matching labels.

All labels have matching images.
